# GridWorld Problem

### Policy Evaluation
This algorithm uses two array for value functions, old and new ones.  
Using one array for value function, also converges (faster) to optimal value function.

In [1]:
import gym
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

In [8]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    valueFunction = [0 for i in range(env.nS)]
    delta = 0.1
    iteration = 0
    while (True):
        iteration = iteration + 1
        vtemp = [0 for i in range(env.nS)]
        delta = 0
        for i in range (env.nS):
            temp = valueFunction[i]
            s = 0
            for a in range(env.nA):
                s+= policy[i][a] *  env.P[i][a][0][0] * (env.P[i][a][0][2] + discount_factor * valueFunction[env.P[i][a][0][1]])
            vtemp[i]= s
#             print("state ",i, "value ", s)
            delta = max(delta, np.abs(s - temp))
#             print("delta \n",delta)
        valueFunction = vtemp
        if (delta<theta):
            print("iteration: ",iteration)
            break
    return valueFunction

In [11]:
random_policy = np.ones([env.nS, env.nA]) / env.nA

v = policy_eval(random_policy, env)
print(v)


iteration:  215
[0.0, -13.99989314905062, -19.999841666134284, -21.999822815040588, -13.99989314905062, -17.99986051722798, -19.999842725281216, -19.999841666134284, -19.99984166613428, -19.999842725281212, -17.99986051722798, -13.99989314905062, -21.999822815040584, -19.999841666134284, -13.999893149050623, 0.0]


### Policy Iteration

In [22]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    policy_stable = True
    while True:
        for i in range(env.nS):
            temp = policy[i]
            print("temp: ",temp)
            state_value = policy_eval_fn(policy, env, discount_factor)[i]
            s = 0
            # prob * (reward + discount * value_next)
            arg =[
                env.P[i][0][0][0] * (env.P[i][0][0][2] +  policy_eval_fn(policy,env,discount_factor)[env.P[i][0][0][1]] * discount_factor),
                env.P[i][1][0][0] * (env.P[i][1][0][2] +  policy_eval_fn(policy,env,discount_factor)[env.P[i][1][0][1]] * discount_factor),
                env.P[i][2][0][0] * (env.P[i][2][0][2] +  policy_eval_fn(policy,env,discount_factor)[env.P[i][2][0][1]] * discount_factor),
                env.P[i][3][0][0] * (env.P[i][3][0][2] +  policy_eval_fn(policy,env,discount_factor)[env.P[i][3][0][1]] * discount_factor) 
                 ]
            policy[i] = np.argmax(arg)
            print(policy[i])
            if (temp != policy[i]):
                policy_stable = False
        if(policy_stable):
            break;
    return policy, policy_eval_fn(policy,env,discount_factor)

In [23]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

temp:  [0.25 0.25 0.25 0.25]
iteration:  215
iteration:  215
iteration:  215
iteration:  215
iteration:  215


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()